# VQE & QAOA Patterns - Code Laboratory

**Section 6: Estimator & VQE/QAOA** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Component | API | Use |
|-----------|-----|-----|
| Ansatz | `QuantumCircuit` + `Parameter` | Variational circuit |
| Hamiltonian | `SparsePauliOp(['ZZ', 'XX'], coeffs)` | Observable |
| Estimator | `StatevectorEstimator()` | Compute ⟨H⟩ |
| Optimize | `scipy.optimize.minimize(cost, x0, method='COBYLA')` | Find minimum |
| Access Energy | `result.fun` | Ground state energy |
| Access Params | `result.x` | Optimal parameters |

## 🧠 Mnemonic: AHEO
**A**nsatz → **H**amiltonian → **E**stimator → **O**ptimize

---

In [ ]:
# ============================================================
# ENVIRONMENT SETUP - Run this first!
# ============================================================

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

def run_vqe(ansatz, H, initial_params, method='COBYLA', max_iter=100, verbose=True):
    """Complete VQE optimization pattern."""
    estimator = StatevectorEstimator()
    iterations = [0]
    
    def cost(params):
        iterations[0] += 1
        qc = ansatz.assign_parameters(params)
        job = estimator.run([(qc, H)])
        energy = job.result()[0].data.evs
        if verbose and iterations[0] % 20 == 0:
            print(f"  Iter {iterations[0]}: E = {energy:.6f}")
        return energy
    
    result = minimize(cost, initial_params, method=method, 
                      options={'maxiter': max_iter})
    return result

print("✅ Environment ready")
print("   VQE Components: Ansatz, Hamiltonian, Estimator, Optimizer")
print("   Mnemonic: AHEO")

---

## 1️⃣ VQE Cost Function Pattern (MEMORIZE!)

### The Pattern Every VQE Uses
```python
def cost_function(params):
    # 1. Bind parameters
    qc = ansatz.assign_parameters(params)
    
    # 2. Run Estimator
    job = estimator.run([(qc, H)])
    result = job.result()
    
    # 3. Extract energy
    energy = result[0].data.evs  # ⚠️ evs plural!
    
    return energy  # ← Minimize this!
```

In [ ]:
# Simple VQE: Find ground state of H = Z
# ========================================
# Expected: Ground state |1⟩, energy = -1

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# 1. ANSATZ - Parameterized circuit
theta = Parameter('θ')
ansatz = QuantumCircuit(1)
ansatz.ry(theta, 0)

print("1. ANSATZ:")
print(ansatz.draw())

# 2. HAMILTONIAN - What we want to minimize
H = SparsePauliOp('Z')
print(f"\n2. HAMILTONIAN: H = Z")

# 3. ESTIMATOR - Computes ⟨H⟩
estimator = StatevectorEstimator()

# 4. COST FUNCTION - Returns energy
def cost(params):
    qc = ansatz.assign_parameters({theta: params[0]})
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs  # ⚠️ .evs (plural!)

# 5. OPTIMIZE - Find minimum
print("\n3-5. OPTIMIZING...")
result = minimize(cost, x0=[np.pi/4], method='COBYLA')

print(f"\n✅ VQE RESULTS:")
print(f"   Optimal θ: {result.x[0]:.4f} rad ≈ {result.x[0]/np.pi:.2f}π")
print(f"   Ground energy: {result.fun:.6f}")
print(f"   Expected: -1.0 at θ = π")

# Verify
print(f"\n📝 Why θ = π?")
print(f"   Ry(π)|0⟩ = |1⟩")
print(f"   ⟨1|Z|1⟩ = -1 (eigenvalue of |1⟩)")

---

## 2️⃣ scipy.optimize.minimize Options

### Common Optimizers (MEMORIZE!)
| Method | Type | Best For |
|--------|------|----------|
| `COBYLA` | Gradient-free | **Default for VQE** |
| `SLSQP` | Gradient-based | Fast, exact |
| `Nelder-Mead` | Gradient-free | Alternative |

```python
from scipy.optimize import minimize
result = minimize(cost_fn, x0, method='COBYLA')

# Access results:
result.fun  # Minimum energy found
result.x    # Optimal parameters
result.nfev # Number of function evaluations
```

In [ ]:
# Two-Qubit VQE with Complex Hamiltonian
# ========================================

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# Hamiltonian: H = ZI + IZ + 0.5*XX (Ising-like)
H = SparsePauliOp.from_list([
    ('ZI', 1.0),
    ('IZ', 1.0),
    ('XX', 0.5)
])
print(f"Hamiltonian: H = ZI + IZ + 0.5*XX")

# Ansatz with ParameterVector (easier for multiple params)
params = ParameterVector('θ', 3)

ansatz = QuantumCircuit(2)
ansatz.ry(params[0], 0)
ansatz.ry(params[1], 1)
ansatz.cx(0, 1)
ansatz.rz(params[2], 1)

print(f"\nAnsatz with 3 parameters:")
print(ansatz.draw())

# Estimator and cost function
estimator = StatevectorEstimator()

def cost(param_values):
    qc = ansatz.assign_parameters(param_values)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# Optimize
np.random.seed(42)
x0 = np.random.random(3)
result = minimize(cost, x0, method='COBYLA', options={'maxiter': 200})

print(f"\n✅ VQE RESULTS:")
print(f"   Ground energy: {result.fun:.6f}")
print(f"   Optimal params: {result.x}")
print(f"   Iterations: {result.nfev}")

---

## 3️⃣ H2 Molecule VQE (Exam Pattern)

### H2 Hamiltonian (MEMORIZE!)
```python
H2 = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)
# Expected ground state energy: ≈ -1.85 Hartree
```

In [ ]:
# H2 Molecule VQE (Common Exam Pattern)
# ======================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# H2 Hamiltonian (Jordan-Wigner encoding)
H2 = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)
print("H2 Molecule Hamiltonian:")
print("H = -1.05*II + 0.39*ZI + 0.39*IZ - 0.01*ZZ + 0.18*XX")
print(f"Number of terms: {len(H2)}")

# Ansatz
theta = Parameter('θ')
ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(theta, 1)
ansatz.cx(0, 1)
ansatz.ry(theta, 0)
ansatz.ry(theta, 1)

# VQE
estimator = StatevectorEstimator()

def cost(params):
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H2)])
    return job.result()[0].data.evs

print("\nOptimizing...")
result = minimize(cost, [0.0], method='COBYLA')

print(f"\n✅ H2 VQE RESULTS:")
print(f"   Ground state energy: {result.fun:.6f} Hartree")
print(f"   Optimal θ: {result.x[0]:.4f}")
print(f"   Expected (exact): ≈ -1.85 Hartree")
print(f"   Error: {abs(result.fun + 1.85):.4f}")

In [ ]:
# VQE + Sampler Verification Pattern
# ====================================
# Common pattern: Run VQE, then verify with Sampler

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# 1. VQE Setup
H = SparsePauliOp('Z')
theta = Parameter('θ')
ansatz = QuantumCircuit(1)
ansatz.ry(theta, 0)

estimator = StatevectorEstimator()

def cost(params):
    qc = ansatz.assign_parameters({theta: params[0]})
    return estimator.run([(qc, H)]).result()[0].data.evs

# 2. Run VQE
result = minimize(cost, x0=[0.5], method='COBYLA')
optimal_theta = result.x[0]

print("VQE Results:")
print(f"  Optimal θ: {optimal_theta:.4f}")
print(f"  Ground energy: {result.fun:.6f}")

# 3. Verify with Sampler (measure the ground state)
optimal_qc = ansatz.assign_parameters({theta: optimal_theta})
optimal_qc.measure_all()

sampler = StatevectorSampler()
job = sampler.run([(optimal_qc,)], shots=1024)
counts = job.result()[0].data.meas.get_counts()

print(f"\n✅ SAMPLER VERIFICATION:")
print(f"  Counts: {counts}")
print(f"  Ground state |1⟩ should dominate (⟨Z⟩=-1)")
print(f"  |1⟩ probability: {counts.get('1', 0)/1024:.1%}")

In [ ]:
# ⭐ COMPLETE VQE PATTERN - MEMORIZE THIS ⭐
# ==========================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# STEP 1: Define Hamiltonian (H)
H = SparsePauliOp.from_list([
    ('ZI', 1.0),
    ('IZ', 1.0),
    ('XX', 0.5)
])

# STEP 2: Create Parameterized Ansatz (A)
num_qubits = 2
theta = [Parameter(f'θ{i}') for i in range(2)]
ansatz = QuantumCircuit(num_qubits)
ansatz.ry(theta[0], 0)
ansatz.ry(theta[1], 1)
ansatz.cx(0, 1)

# STEP 3: Create Estimator (E)
estimator = StatevectorEstimator()
iteration_count = 0

# STEP 4: Define Cost Function (O)
def cost_function(params):
    global iteration_count
    iteration_count += 1
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# STEP 5: Optimize
print("Running VQE optimization...")
initial_params = np.random.random(2) * np.pi
result = minimize(
    cost_function,
    initial_params,
    method='COBYLA',
    options={'maxiter': 100}
)

# STEP 6: Extract Results
print(f"\n{'='*50}")
print("VQE RESULTS")
print(f"{'='*50}")
print(f"Ground state energy: {result.fun:.6f}")
print(f"Optimal parameters: θ0={result.x[0]:.4f}, θ1={result.x[1]:.4f}")
print(f"Total iterations: {iteration_count}")

# EXAM KEY POINTS:
print(f"\n{'='*50}")
print("EXAM CHECKLIST (AHEO):")
print("  A - Ansatz: ansatz.assign_parameters(params)")
print("  H - Hamiltonian: SparsePauliOp.from_list([...])")
print("  E - Estimator: estimator.run([(qc, H)])")
print("  O - Optimizer: minimize(cost, x0, method='COBYLA')")
print(f"{'='*50}")
print("  result.fun  → ground state energy")
print("  result.x    → optimal parameters")

In [ ]:
# VQE with Ising Model Hamiltonian
# =================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# Ising model: H = ZZ - ZI - IZ
H = SparsePauliOp(['ZZ', 'ZI', 'IZ'], [1.0, -1.0, -1.0])
print(f'Ising Hamiltonian: H = ZZ - ZI - IZ')

# Two-parameter ansatz
theta, phi = Parameter('θ'), Parameter('φ')
ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(phi, 1)
ansatz.cx(0, 1)
ansatz.ry(theta, 0)

# VQE
estimator = StatevectorEstimator()
iteration_count = [0]

def cost_function(params):
    iteration_count[0] += 1
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

print('\nOptimizing...')
result = minimize(cost_function, [0.0, 0.0], method='COBYLA', options={'maxiter': 100})

print(f'\n✅ VQE RESULTS:')
print(f'   Ground state energy: {result.fun:.6f}')
print(f'   Optimal θ={result.x[0]:.4f}, φ={result.x[1]:.4f}')
print(f'   Iterations: {iteration_count[0]}')

In [ ]:
# Optimizer Comparison: COBYLA vs SLSQP vs Nelder-Mead
# =====================================================

from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Setup
H = SparsePauliOp(['ZI', 'IZ', 'XX'], [1.0, 1.0, 0.5])
theta = Parameter('θ')
qc = QuantumCircuit(2)
qc.ry(theta, 0)
qc.ry(theta, 1)
qc.cx(0, 1)

estimator = StatevectorEstimator()

def cost(params):
    return estimator.run([(qc.assign_parameters(params), H)]).result()[0].data.evs

# Compare optimizers
methods = ['COBYLA', 'SLSQP', 'Nelder-Mead']
print('Optimizer Comparison:')
print('='*50)
for method in methods:
    result = minimize(cost, [0.5], method=method, options={'maxiter': 50})
    print(f'{method:12s}: E = {result.fun:.6f}, θ = {result.x[0]:.4f}')

print(f'\n📊 OPTIMIZER EXAM NOTES:')
print(f'   COBYLA    - Most common in VQE (gradient-free, handles constraints)')
print(f'   SLSQP     - Uses gradients (faster if gradients available)')
print(f'   Nelder-Mead - Simplex method (robust, no gradients)')
print(f'   ⚠️ All gradient-free methods work with noisy quantum hardware')

In [ ]:
# QAOA Pattern: MaxCut on Triangle Graph
# =======================================
# QAOA = Special VQE with structured ansatz

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

print('QAOA for MaxCut (Triangle Graph)')
print('='*50)
print('Graph: 3 nodes, edges (0,1), (0,2), (1,2)')

# Cost Hamiltonian: edges
H_cost = SparsePauliOp(['ZZI', 'ZIZ', 'IZZ'], [1.0, 1.0, 1.0])

def qaoa_circuit(gamma, beta, p=1):
    """QAOA circuit with p layers"""
    qc = QuantumCircuit(3)
    qc.h([0, 1, 2])  # Initial: equal superposition
    
    for _ in range(p):
        # Cost layer (problem-specific)
        qc.rzz(2*gamma, 0, 1)  # Edge (0,1)
        qc.rzz(2*gamma, 0, 2)  # Edge (0,2)
        qc.rzz(2*gamma, 1, 2)  # Edge (1,2)
        
        # Mixer layer (standard)
        qc.rx(2*beta, 0)
        qc.rx(2*beta, 1)
        qc.rx(2*beta, 2)
    return qc

# QAOA optimization
estimator = StatevectorEstimator()
iteration = [0]

def qaoa_cost(params):
    gamma, beta = params
    qc = qaoa_circuit(gamma, beta)
    iteration[0] += 1
    return estimator.run([(qc, H_cost)]).result()[0].data.evs

print('\nOptimizing QAOA...')
result = minimize(qaoa_cost, [0.5, 0.5], method='COBYLA', options={'maxiter': 50})

print(f'\n✅ QAOA RESULTS:')
print(f'   Optimal energy: {result.fun:.4f}')
print(f'   Optimal γ={result.x[0]:.4f}, β={result.x[1]:.4f}')
print(f'   Iterations: {iteration[0]}')

print(f'\n📌 QAOA KEY POINTS:')
print(f'   • Cost layer: rzz gates (problem-specific)')
print(f'   • Mixer layer: rx gates (standard)')
print(f'   • Parameters: γ (gamma) for cost, β (beta) for mixer')
print(f'   • QAOA = VQE with structured ansatz')

In [ ]:
# 🏋️ CODE CHALLENGE 1: H2 Molecule VQE
# =====================================
# Complete the VQE for H2 molecule

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# H2 Hamiltonian
H = SparsePauliOp(["II", "ZI", "IZ", "ZZ", "XX"], [-1.05, 0.39, 0.39, -0.01, 0.18])

# TODO: Create ansatz with 2 parameters
theta, phi = Parameter('θ'), Parameter('φ')
ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(phi, 1)
ansatz.cx(0, 1)

# TODO: Implement VQE
estimator = StatevectorEstimator()

def cost(params):
    qc = ansatz.assign_parameters(params)
    return estimator.run([(qc, H)]).result()[0].data.evs

result = minimize(cost, [0.0, 0.0], method='COBYLA')

# Verify
assert result.fun < -1.0, "Energy should be negative"
print(f"✅ Ground state energy: {result.fun:.4f} Hartree")
print(f"   Expected: ≈ -1.85 Hartree")

In [ ]:
# 🏋️ CODE CHALLENGE 2: Multi-Layer VQE Comparison
# ================================================
# Compare VQE accuracy with different circuit depths

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

H = SparsePauliOp(['ZZ', 'XX', 'YY'], [1.0, 0.5, 0.5])

def create_ansatz(n_layers):
    """Create n-layer ansatz"""
    params = ParameterVector('θ', n_layers * 2)
    qc = QuantumCircuit(2)
    for i in range(n_layers):
        qc.ry(params[2*i], 0)
        qc.ry(params[2*i+1], 1)
        qc.cx(0, 1)
    return qc, params

estimator = StatevectorEstimator()
print('Circuit Depth Impact on VQE:')
print('='*50)

results = []
for n_layers in [1, 2, 3]:
    ansatz, params = create_ansatz(n_layers)
    
    def cost(param_values, ansatz=ansatz):
        qc = ansatz.assign_parameters(param_values)
        return estimator.run([(qc, H)]).result()[0].data.evs
    
    result = minimize(cost, np.zeros(n_layers * 2), method='COBYLA', options={'maxiter': 100})
    results.append({'layers': n_layers, 'energy': result.fun})
    print(f'  {n_layers} layer(s): E = {result.fun:.6f}')

# Verify: more layers should give equal or better energy
assert results[1]['energy'] <= results[0]['energy'] + 0.1, "2 layers should improve on 1 layer"
print(f"\n✅ More layers = better approximation")
print(f"   Best: {min(results, key=lambda x: x['energy'])['layers']} layers")

In [ ]:
# 🏋️ CODE CHALLENGE 3: QAOA MaxCut
# ==================================
# Implement QAOA for 4-node path graph: 0-1-2-3

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# Path graph edges: (0,1), (1,2), (2,3)
H_cost = SparsePauliOp(['ZZII', 'IZZI', 'IIZZ'], [1.0, 1.0, 1.0])

def qaoa_circuit(gamma, beta):
    qc = QuantumCircuit(4)
    qc.h([0, 1, 2, 3])  # Initial superposition
    
    # Cost layer: rzz for each edge
    qc.rzz(2*gamma, 0, 1)
    qc.rzz(2*gamma, 1, 2)
    qc.rzz(2*gamma, 2, 3)
    
    # Mixer layer: rx for each qubit
    qc.rx(2*beta, 0)
    qc.rx(2*beta, 1)
    qc.rx(2*beta, 2)
    qc.rx(2*beta, 3)
    return qc

estimator = StatevectorEstimator()

def qaoa_cost(params):
    qc = qaoa_circuit(params[0], params[1])
    return estimator.run([(qc, H_cost)]).result()[0].data.evs

result = minimize(qaoa_cost, [0.5, 0.5], method='COBYLA')

# Verify
assert result.fun < 2.0, "QAOA should find a good solution"
print(f"✅ QAOA MaxCut Results:")
print(f"   Optimal energy: {result.fun:.4f}")
print(f"   Optimal γ={result.x[0]:.4f}, β={result.x[1]:.4f}")